In [4]:
endpoint = 'https://b22lang.cognitiveservices.azure.com/'
key = '09473493a2824714b9ac54e9396026b6'
project_name = 'customnerdemo'
deployment_name = 'cnerdemodeploy'


def sample_recognize_custom_entities() -> None:
    # [START recognize_custom_entities]
    import os
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient

    path_to_sample_document = './Test 1.txt'

    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key),
    )

    with open(path_to_sample_document) as fd:
        document = [fd.read()]

    poller = text_analytics_client.begin_recognize_custom_entities(
        document,
        project_name=project_name,
        deployment_name=deployment_name
    )

    document_results = poller.result()
    for custom_entities_result in document_results:
        if custom_entities_result.kind == "CustomEntityRecognition":
            for entity in custom_entities_result.entities:
                print(
                    "Entity '{}' has category '{}' with confidence score of '{}'".format(
                        entity.text, entity.category, entity.confidence_score
                    )
                )
        elif custom_entities_result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                custom_entities_result.error.code, custom_entities_result.error.message
                )
            )
    # [END recognize_custom_entities]


if __name__ == "__main__":
    sample_recognize_custom_entities()

Entity '10/18/2019' has category 'Date' with confidence score of '1.0'
Entity 'Casey Jensen' has category 'BorrowerName' with confidence score of '1.0'
Entity '2469 Pennsylvania Avenue' has category 'BorrowerAddress' with confidence score of '0.99'
Entity 'New Brunswick' has category 'BorrowerCity' with confidence score of '0.95'
Entity 'New Jersey' has category 'BorrowerState' with confidence score of '0.81'
Entity 'Hollie Rees' has category 'LenderName' with confidence score of '1.0'
Entity '42 Gladwell Street' has category 'LenderAddress' with confidence score of '1.0'
Entity 'Memphis' has category 'LenderCity' with confidence score of '1.0'
Entity 'Tennessee' has category 'LenderState' with confidence score of '1.0'
Entity 'one hundred ninety-two thousand nine hundred eighty-nine Dollars' has category 'LoanAmountWords' with confidence score of '1.0'
Entity '$192,989.00' has category 'LoanAmountNumbers' with confidence score of '1.0'
Entity '5%' has category 'Interest' with confiden

In [ ]:
#이게 Rest api인지 아니면 뭐 다른건지 모르겠네
#방법이2개라던데
import os
import requests
import json
import time
 
def recognize_custom_entities():
    # 샘플을 실행하기 전에 환경 변수를 자신의 값으로 설정합니다.
    endpoint = os.getenv("AZURE_LANGUAGE_ENDPOINT", "https://fletaaiservicesgroup.cognitiveservices.azure.com/language/analyze-text/jobs?api-version=2022-10-01-preview")
    subscription_key = os.getenv("AZURE_LANGUAGE_KEY", "e2895d88fc1445baa39a971e40c3c17d")
    project_name = os.getenv("CUSTOM_ENTITIES_PROJECT_NAME", "customnerdomo")
    deployment_name = os.getenv("CUSTOM_ENTITIES_DEPLOYMENT_NAME", "cnerdendeply")
    path_to_sample_document = "Test 1.txt"
 
    # 문서를 읽습니다.
    with open(path_to_sample_document, 'r') as file:
        document_text = file.read()
 
    # 요청 페이로드를 준비합니다.
    payload = {
        "tasks": [
            {
                "kind": "CustomEntityRecognition",
                "parameters": {
                    "projectName": project_name,
                    "deploymentName": deployment_name,
                    "stringIndexType": "TextElement_v8"
                }
            }
        ],
        "displayName": "CustomTextPortal_CustomEntityRecognition",
        "analysisInput": {
            "documents": [
                {
                    "id": "document_CustomEntityRecognition",
                    "text": document_text,
                    "language": "en"  # 문서의 언어가 다르면 여기를 변경하세요.
                }
            ]
        }
    }
 
    # 헤더를 설정합니다.
    headers = {
        "Ocp-Apim-Subscription-Key": subscription_key,
        "Content-Type": "application/json"
    }
 
    # 요청을 보냅니다.
    response = requests.post(endpoint, headers=headers, data=json.dumps(payload))
 
    if response.status_code == 202:
        operation_location = response.headers["operation-location"]
        print(f"작업이 수락되었습니다. 상태를 확인하기 위해 {operation_location}을(를) 주기적으로 확인하십시오.")
 
        # 주기적으로 상태를 확인하여 작업이 완료될 때까지 대기합니다.
        while True:
            status_response = requests.get(operation_location, headers=headers)
            status_result = status_response.json()
 
            if status_response.status_code != 200:
                print(f"상태 확인 오류: {status_response.status_code} - {status_response.text}")
                break
 
            if status_result["status"] == "succeeded":
                print("사용자 정의 엔터티 인식 결과:")
                for task in status_result['tasks']['items']:
                    if task['status'] == 'succeeded':
                        for entity_result in task['results']['documents']:
                            for entity in entity_result['entities']:
                                print(f"엔터티 '{entity['text']}'는 카테고리 '{entity['category']}'를 가지며 신뢰 점수는 '{entity['confidenceScore']}'입니다.")
                    else:
                        print(f"작업 실패, 오류: {task['results']['errors']}")
                break
            elif status_result["status"] == "failed":
                print(f"작업 실패: {status_result}")
                break
            else:
                print(f"작업 상태: {status_result['status']}... 대기 중...")
                time.sleep(5)  # 5초 대기 후 상태를 다시 확인합니다.
    else:
        print(f"오류: {response.status_code} - {response.text}")
 
if __name__ == "__main__":
    recognize_custom_entities()
